---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('../materials/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    # Return mean of target and multiply by 100
    return spam_data['target'].mean() * 100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    # Fit CountVectorizer into X_train
    vect = CountVectorizer().fit(X_train)

    # Return longest token in list of tokens
    return max(vect.get_feature_names(), key=len)

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    # Fit Xtrain into CountVectorizer
    vect = CountVectorizer().fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Train classifier
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    # Predict X_test (after transforming)
    pred = model.predict(vect.transform(X_test))
    
    # Return AUC score
    return roc_auc_score(y_test, pred)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    # Fit Xtrain into TfidfVectorizer
    vect = TfidfVectorizer().fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Compute the maximum tf-idf value per feature (token)
    tfidf_values = X_train_vectorized.max(0).toarray()[0]
    
    # Sort the indexes by ascending order
    sorted_tfidf_index = tfidf_values.argsort()
    
    # Get feature names
    feature_names = np.array(vect.get_feature_names())
    
    # Get top-20 features with largest and smallest tf-idf values
    largest_series = pd.Series(data = tfidf_values[sorted_tfidf_index[-20:]], 
                               index = feature_names[sorted_tfidf_index[-20:]]).sort_values(ascending = False)
    
    smallest_series = pd.Series(data = tfidf_values[sorted_tfidf_index[:20]], 
                               index = feature_names[sorted_tfidf_index[:20]])
    
    return (smallest_series, largest_series)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [10]:
def answer_five():
    
    # Fit Xtrain into TfidfVectorizer
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Train classifier
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    # Predict X_test (after transforming)
    pred = model.predict(vect.transform(X_test))
    
    # Return AUC score
    return roc_auc_score(y_test, pred)

In [11]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [12]:
def answer_six():
    
    # Compute length of each document
    spam_data['length'] = spam_data['text'].apply(lambda x: len(x))
    
    # Compute average lenght per class
    average_values = spam_data.groupby('target')['length'].mean().values
    
    return (average_values[0], average_values[1])

In [13]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [14]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [15]:
from sklearn.svm import SVC

def answer_seven():
    
    # Fit Xtrain into TfidfVectorizer
    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Concat length of each document into the training set
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(x)))
    
    # Train classifier
    model = SVC(C = 10000)
    model.fit(X_train_vectorized, y_train)
    
    # Predict X_test (after transforming)
    pred = model.predict(add_feature(vect.transform(X_test), X_test.apply(lambda x: len(x))))
    
    # Return AUC score
    return roc_auc_score(y_test, pred)

In [16]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [17]:
import re

def answer_eight():
    
    # Get the number of digits by computing the length of each string after removing non-digit characters
    spam_data['digits_number'] = spam_data['text'].apply(lambda x: len(re.sub('[^0-9]', '', x)))
    
    # Compute average number of digits per class
    average_values = spam_data.groupby('target')['digits_number'].mean().values
    
    return (average_values[0], average_values[1])

In [18]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [19]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    # Fit Xtrain into TfidfVectorizer
    vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Add extra features to the training set
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(x)))
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(re.sub('[^0-9]', '', x))))
    
    # Train classifier
    model = LogisticRegression(C = 100)
    model.fit(X_train_vectorized, y_train)
    
    # Transform X_test
    X_test_vectorized = vect.transform(X_test)
    
    # Add extra features to the test set
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda x: len(x)))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda x: len(re.sub('[^0-9]', '', x))))
    
    # Predict X_test (after transforming)
    pred = model.predict(X_test_vectorized)
    
    # Return AUC score
    return roc_auc_score(y_test, pred)

In [20]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [21]:
def answer_ten():
    
    # Get the number of non-word characters
    spam_data['non_word_number'] = spam_data['text'].apply(lambda x: len(re.sub('\w', '', x)))
    
    # Compute average number of non-word characters per class
    average_values = spam_data.groupby('target')['non_word_number'].mean().values
    
    return (average_values[0], average_values[1])

In [22]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [23]:
def answer_eleven():
    
    # Fit Xtrain into CountVectorizer
    vect = CountVectorizer(min_df = 5, ngram_range = (2,5), analyzer='char_wb').fit(X_train)
    
    # Transform X_train to a numeric representation
    X_train_vectorized = vect.transform(X_train)
    
    # Add extra features to the training set
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(x)))
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(re.sub('[^0-9]', '', x))))
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda x: len(re.sub('\w', '', x))))
    
    # Train classifier
    model = LogisticRegression(C = 100)
    model.fit(X_train_vectorized, y_train)
    
    # Transform X_test
    X_test_vectorized = vect.transform(X_test)
    
    # Add extra features to the test set
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda x: len(x)))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda x: len(re.sub('[^0-9]', '', x))))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda x: len(re.sub('\w', '', x))))
    
    # Predict X_test (after transforming)
    pred = model.predict(X_test_vectorized)
    
    # Get feature names
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    # Compute coefficients and sort indexes
    sorted_coef_index = model.coef_[0].argsort()
    
    # Retrieve feature names
    smallest = feature_names[sorted_coef_index[:10]]
    largest =  feature_names[sorted_coef_index[-10:]]
    
    # Return 
    return (roc_auc_score(y_test, pred), smallest, largest[::-1])

In [24]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U19'),
 array(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww',
        'ar'], 
       dtype='<U19'))